<a href="https://colab.research.google.com/github/dpressel/dlss-tutorial/blob/master/3_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part III: Fine-tuning a pre-trained model

In the last section, we looked at using a biLM networks layers as embeddings for our classification model.  In that approach, we maintain the exact same model architecture as before, but just switching our word embeddings out for context embeddings (or, more commonly, using them in concert).

The paper [Improving Language Understanding
by Generative Pre-Training](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf) (Radford et al 2018) explored a different approach, much more similar to what is typically done in computer vision.  In fine-tuning, we reuse the network architecture and simply replace the head.  We dont use any model specific architecture anymore, just a final layer.  There is an accompanying blog post [here](https://openai.com/blog/language-unsupervised/).  The image below is borrowed from that blog post

![alt text](https://openai.com/content/images/2018/06/zero-shot-transfer@2x.png)

As we can see from the images, these models can rapidly improve our downstream performance with very limited fine-tuning supervision.

## The Transformer

The original Transformer is an all-attention encoder-decoder model.  It is described at a high-level in [this Google AI post](https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html).  This not really low-level enough to understand how it actually works.

Here is an image of the model architecture for a Transformer:

![Transformer Architecture](http://nlp.seas.harvard.edu/images/the-annotated-transformer_14_0.png)

The reference implementation from Google is the [tensor2tensor repository](https://github.com/tensorflow/tensor2tensor/tree/master/tensor2tensor).  There is a lot going on in that codebase, which some people may find hard to follow.

If you want to understand Transformers better, there is a terrific blog post called [The Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html) (Rush 2018) where you can see how to code up a Transformer from scratch to do Neural Machine Translation (NMT) while following along with the paper.  

In versions used in practice, there are slight differences from the actual image, most notably, that layer norm is performed first.  Also, in a causal LM pre-training setting, we have no need for the decoder, which simplifies our architecture substantially.  

### Some implementation details

#### Multi-headed attention and positional embeddings

Multi-headed attention is one of the key innovations of the Transformer.  The idea was to allow each attention head to learn different relations between words as seen in this image from Google's blog:

![MHA](https://1.bp.blogspot.com/-AVGK0ApREtk/WaiAuzddKVI/AAAAAAAAB_A/WPV5ropBU-cxrcMpqJBFHg73K9NX4vywwCLcBGAs/s1600/image2.png)

For language modeling, as in the case of GPT, the decoder is entirely discarded, leaving only a masked self-attention in the encoder (this prevents us from seeing the future as we predict).

![MHA Architecture](http://nlp.seas.harvard.edu/images/the-annotated-transformer_33_0.png)


#### Positional embeddings

To eliminate auto-regressive (RNN) models from the transformer, positional embeddings need to be created and added to the word embeddings.  Otherwise, during attention there would be no way to account for word position.  This is described in the blog post mentioned above.  There are 2 basic ways to support positional embeddings, sinusoidal and learned positional embeddings.


#### A Transformer Encoder Layer

Here is code adapted from [Baseline](https://github.com/dpressel/baseline) that implements a Transformer block used in a GPT-like architecture


```python

class TransformerEncoder(nn.Module):
    def __init__(self, num_heads, d_model, pdrop, scale=True, activation_type='relu', d_ff=None):
        """
        :param num_heads (`int`): the number of heads for self-attention
        :param d_model (`int`): The model dimension size
        :param pdrop (`float`): The dropout probability
        :param scale (`bool`): Whether we are doing scaled dot-product attention
        :param activation_type: What activation type to use
        :param d_ff: The feed forward layer size
        """
        super(TransformerEncoder, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff if d_ff is not None else num_heads * d_model
        self.self_attn = MultiHeadedAttention(num_heads, d_model, pdrop, scale=scale)
        self.ffn = nn.Sequential(pytorch_linear(self.d_model, self.d_ff),
                                 pytorch_activation(activation_type),
                                 pytorch_linear(self.d_ff, self.d_model))
        self.ln1 = nn.LayerNorm(self.d_model, eps=1e-12)
        self.ln2 = nn.LayerNorm(self.d_model, eps=1e-12)
        self.dropout = nn.Dropout(pdrop)

    def forward(self, x, mask=None):
        """
        :param x: the inputs
        :param mask: a mask for the inputs
        :return: the encoder output
        """
        # Builtin Attention mask
        x = self.ln1(x)
        h = self.self_attn(x, x, x, mask)
        x = x + self.dropout(h)

        x = self.ln2(x)
        x = x + self.dropout(self.ffn(x))
        return x

```

#### Multi-Headed Attention in PyTorch

This operation is now built into PyTorch, with the limitation that only scaled-dot product attention is supported (Tensor2Tensor, Google's reference implementation supports both).  The code above does not use that module since it supports both scaled and unscaled attention




There is also an end-to-end example using the Baseline code above to train a GPT-like LM using the code above in PyTorch:

https://github.com/dpressel/baseline/blob/master/api-examples/pretrain-transformer-lm.py



## BERT

For this section of the tutorial, we are going to fine-tune BERT [Devlin et al 2018](https://arxiv.org/abs/1810.04805), a transformer architecture that replaces the causal LM objective with 2 new objectives:

1. Masking out words with some probability, predict the missing words

![MLM](https://2.bp.blogspot.com/-pNxcHHXNZg0/W9iv3evVyOI/AAAAAAAADfA/KTSvKXNzzL0W8ry28PPl7nYI1CG_5WuvwCLcBGAs/s1600/f1.png)

2. Given 2 adjacent sentences, predict if the second sentence follows the first

![NSP](https://4.bp.blogspot.com/-K_7yu3kjF18/W9iv-R-MnyI/AAAAAAAADfE/xUwR_G1iTY0vq9X-Z3LnW5t4NLS9BQzdgCLcBGAs/s1600/f2.png)

From an architecture diagram, [this blog post announcing BERT](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html) notes the differences:

![BERT vs GPT and ELMo](https://1.bp.blogspot.com/-RLAbr6kPNUo/W9is5FwUXmI/AAAAAAAADeU/5y9466Zoyoc96vqLjbruLK8i_t8qEdHnQCLcBGAs/s1600/image3.png)

Our model will simply build on the existing model architecture with a single transformation layer to the output number of classes.  BERT is [open source](https://github.com/google-research/bert) but the code is in TensorFlow, and since this tutorial is written in PyTorch, we need a different solution.  We will use the [Hugging Face Transformer codebase](https://github.com/huggingface/pytorch-pretrained-BERT) as our API -- it can read in the original Google-trained weights.

In [0]:
!pip install pytorch-pretrained-bert


     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 655kB 42.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import io
import os
import re
import codecs
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel


### Tokenization in BERT

In the last sequence, we talked about how ELMo biLMs can limit their parameters while accounting for unseen words using character-compositional word embeddings.  This technique is very powerful, but its also slow.  It is common in NMT to use some sort of sub-word encoding that limits the vocabulary size, but allows us to not have unattested words.  The `tensor2tensor` codebase, for example, creates an invertible encoding for words into sub-tokens with a limited voabulary.  The tokenizer is built from a corpus upfront and stored in a file, and then can be used to encode text.

There are 4 phases in this algorithm described int the tensor2tensor codebase:


    1. Tokenize into a list of tokens.  Each token is a unicode string of either
      all alphanumeric characters or all non-alphanumeric characters.  We drop
      tokens consisting of a single space that are between two alphanumeric
      tokens.
    2. Escape each token.  This escapes away special and out-of-vocabulary
      characters, and makes sure that each token ends with an underscore, and
      has no other underscores.
    3. Represent each escaped token as a the concatenation of a list of subtokens
      from the limited vocabulary.  Subtoken selection is done greedily from
      beginning to end.  That is, we construct the list in order, always picking
      the longest subtoken in our vocabulary that matches a prefix of the
      remaining portion of the encoded token.
    4. Concatenate these lists.  This concatenation is invertible due to the
      fact that the trailing underscores indicate when one list is finished.


On each iteration, all the words are segmented and counted, and the ones with high enough counts are stored in the new vocabulary.

We can access Google's BERT Tokenizer via the Hugging Face API

In [0]:


def whitespace_tokenizer(words):
    return words.split() 

def sst2_tokenizer(words):
    REPLACE = { "'s": " 's ",
                "'ve": " 've ",
                "n't": " n't ",
                "'re": " 're ",
                "'d": " 'd ",
                "'ll": " 'll ",
                ",": " , ",
                "!": " ! ",
                }
    words = words.lower()
    words = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", words)
    for k, v in REPLACE.items():
            words = words.replace(k, v)
    return [w.strip() for w in words.split()]

BERT_TOKENIZER = BertTokenizer.from_pretrained('bert-base-uncased')
BERT_MODEL = BertModel.from_pretrained('bert-base-uncased')
def bert_tokenizer(words, pretokenizer=whitespace_tokenizer):
    subwords = ['[CLS]']
    for word in pretokenizer(words):
        if word == '<unk>':
            subword = '[UNK]'
        else:
            subword = BERT_TOKENIZER.tokenize(word)
        subwords += subword
    return subwords + ['[SEP]']

def bert_vectorizer(sentence):
    return BERT_TOKENIZER.convert_tokens_to_ids(sentence)
    #return [BERT_TOKENIZER.vocab.get(subword, BERT_TOKENIZER.vocab['[PAD]']) for subword in sentence]



Our model this time around is very simple.  It has an output linear layer that comes from pooled output from BERT

In [0]:

class FineTuneClassifier(nn.Module):

    def __init__(self, base_model, num_classes, embed_dim, hidden_units=[]):
        super().__init__()
        self.base_model = base_model
        input_units = embed_dim
        output_units = embed_dim
        sequence = []
        for h in hidden_units:
            sequence.append(nn.Linear(input_units, h))
            input_units = h
            output_units = h
            
        sequence.append(nn.Linear(output_units, num_classes))
        self.outputs = nn.Sequential(*sequence)

    def forward(self, inputs):
        x, lengths = inputs
        
        input_mask = torch.zeros(x.shape, device=x.device, dtype=torch.long).masked_fill(x != 0, 1)
        input_type_ids = torch.zeros(x.shape, device=x.device, dtype=torch.long)
        _, pooled = self.base_model(x, token_type_ids=input_type_ids, attention_mask=input_mask)
        
        stacked = self.outputs(pooled)
        return F.log_softmax(stacked, dim=-1)

All the rest of our code comes from the previous sections

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Tuple
import os
import io
import re
import codecs
import numpy as np
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

class LSTMClassifier(nn.Module):

    def __init__(self, embeddings, num_classes, embed_dims, rnn_units, rnn_layers=1, dropout=0.5, hidden_units=[]):
        super().__init__()
        self.embeddings = embeddings
        self.dropout = nn.Dropout(dropout)
        self.rnn = torch.nn.LSTM(embed_dims,
                                 rnn_units,
                                 rnn_layers,
                                 dropout=dropout,
                                 bidirectional=False,
                                 batch_first=False)
        nn.init.orthogonal_(self.rnn.weight_hh_l0)
        nn.init.orthogonal_(self.rnn.weight_ih_l0)
        sequence = []
        input_units = rnn_units
        output_units = rnn_units
        for h in hidden_units:
            sequence.append(nn.Linear(input_units, h))
            input_units = h
            output_units = h
            
        sequence.append(nn.Linear(output_units, num_classes))
        self.outputs = nn.Sequential(*sequence)
        
        
    def forward(self, inputs):
        one_hots, lengths = inputs
        embed = self.dropout(self.embeddings(one_hots))
        embed = embed.transpose(0, 1)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embed, lengths.tolist())
        _, hidden = self.rnn(packed)
        hidden = hidden[0].view(hidden[0].shape[1:])
        linear = self.outputs(hidden)
        return F.log_softmax(linear, dim=-1)

class ConfusionMatrix:
    """Confusion matrix with metrics

    This class accumulates classification output, and tracks it in a confusion matrix.
    Metrics are available that use the confusion matrix
    """
    def __init__(self, labels):
        """Constructor with input labels

        :param labels: Either a dictionary (`k=int,v=str`) or an array of labels
        """
        if type(labels) is dict:
            self.labels = []
            for i in range(len(labels)):
                self.labels.append(labels[i])
        else:
            self.labels = labels
        nc = len(self.labels)
        self._cm = np.zeros((nc, nc), dtype=np.int)

    def add(self, truth, guess):
        """Add a single value to the confusion matrix based off `truth` and `guess`

        :param truth: The real `y` value (or ground truth label)
        :param guess: The guess for `y` value (or assertion)
        """

        self._cm[truth, guess] += 1

    def __str__(self):
        values = []
        width = max(8, max(len(x) for x in self.labels) + 1)
        for i, label in enumerate([''] + self.labels):
            values += ["{:>{width}}".format(label, width=width+1)]
        values += ['\n']
        for i, label in enumerate(self.labels):
            values += ["{:>{width}}".format(label, width=width+1)]
            for j in range(len(self.labels)):
                values += ["{:{width}d}".format(self._cm[i, j], width=width + 1)]
            values += ['\n']
        values += ['\n']
        return ''.join(values)

    def save(self, outfile):
        ordered_fieldnames = OrderedDict([("labels", None)] + [(l, None) for l in self.labels])
        with open(outfile, 'w') as f:
            dw = csv.DictWriter(f, delimiter=',', fieldnames=ordered_fieldnames)
            dw.writeheader()
            for index, row in enumerate(self._cm):
                row_dict = {l: row[i] for i, l in enumerate(self.labels)}
                row_dict.update({"labels": self.labels[index]})
                dw.writerow(row_dict)

    def reset(self):
        """Reset the matrix
        """
        self._cm *= 0

    def get_correct(self):
        """Get the diagonals of the confusion matrix

        :return: (``int``) Number of correct classifications
        """
        return self._cm.diagonal().sum()

    def get_total(self):
        """Get total classifications

        :return: (``int``) total classifications
        """
        return self._cm.sum()

    def get_acc(self):
        """Get the accuracy

        :return: (``float``) accuracy
        """
        return float(self.get_correct())/self.get_total()

    def get_recall(self):
        """Get the recall

        :return: (``float``) recall
        """
        total = np.sum(self._cm, axis=1)
        total = (total == 0) + total
        return np.diag(self._cm) / total.astype(float)

    def get_support(self):
        return np.sum(self._cm, axis=1)

    def get_precision(self):
        """Get the precision
        :return: (``float``) precision
        """

        total = np.sum(self._cm, axis=0)
        total = (total == 0) + total
        return np.diag(self._cm) / total.astype(float)

    def get_mean_precision(self):
        """Get the mean precision across labels

        :return: (``float``) mean precision
        """
        return np.mean(self.get_precision())

    def get_weighted_precision(self):
        return np.sum(self.get_precision() * self.get_support())/float(self.get_total())

    def get_mean_recall(self):
        """Get the mean recall across labels

        :return: (``float``) mean recall
        """
        return np.mean(self.get_recall())

    def get_weighted_recall(self):
        return np.sum(self.get_recall() * self.get_support())/float(self.get_total())

    def get_weighted_f(self, beta=1):
        return np.sum(self.get_class_f(beta) * self.get_support())/float(self.get_total())

    def get_macro_f(self, beta=1):
        """Get the macro F_b, with adjustable beta (defaulting to F1)

        :param beta: (``float``) defaults to 1 (F1)
        :return: (``float``) macro F_b
        """
        if beta < 0:
            raise Exception('Beta must be greater than 0')
        return np.mean(self.get_class_f(beta))

    def get_class_f(self, beta=1):
        p = self.get_precision()
        r = self.get_recall()

        b = beta*beta
        d = (b * p + r)
        d = (d == 0) + d

        return (b + 1) * p * r / d

    def get_f(self, beta=1):
        """Get 2 class F_b, with adjustable beta (defaulting to F1)

        :param beta: (``float``) defaults to 1 (F1)
        :return: (``float``) 2-class F_b
        """
        p = self.get_precision()[1]
        r = self.get_recall()[1]
        if beta < 0:
            raise Exception('Beta must be greater than 0')
        d = (beta*beta * p + r)
        if d == 0:
            return 0
        return (beta*beta + 1) * p * r / d

    def get_all_metrics(self):
        """Make a map of metrics suitable for reporting, keyed by metric name

        :return: (``dict``) Map of metrics keyed by metric names
        """
        metrics = {'acc': self.get_acc()}
        # If 2 class, assume second class is positive AKA 1
        if len(self.labels) == 2:
            metrics['precision'] = self.get_precision()[1]
            metrics['recall'] = self.get_recall()[1]
            metrics['f1'] = self.get_f(1)
        else:
            metrics['mean_precision'] = self.get_mean_precision()
            metrics['mean_recall'] = self.get_mean_recall()
            metrics['macro_f1'] = self.get_macro_f(1)
            metrics['weighted_precision'] = self.get_weighted_precision()
            metrics['weighted_recall'] = self.get_weighted_recall()
            metrics['weighted_f1'] = self.get_weighted_f(1)
        return metrics

    def add_batch(self, truth, guess):
        """Add a batch of data to the confusion matrix

        :param truth: The truth tensor
        :param guess: The guess tensor
        :return:
        """
        for truth_i, guess_i in zip(truth, guess):
            self.add(truth_i, guess_i)

class Trainer:
    def __init__(self, optimizer: torch.optim.Optimizer):
        self.optimizer = optimizer

    def run(self, model, labels, train, loss, batch_size): 
        model.train()       
        train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

        cm = ConfusionMatrix(labels)

        for batch in train_loader:
            loss_value, y_pred, y_actual = self.update(model, loss, batch)
            _, best = y_pred.max(1)
            yt = y_actual.cpu().int().numpy()
            yp = best.cpu().int().numpy()
            cm.add_batch(yt, yp)

        print(cm.get_all_metrics())
        return cm
    
    def update(self, model, loss, batch):
        self.optimizer.zero_grad()
        x, lengths, y = batch
        lengths, perm_idx = lengths.sort(0, descending=True)
        x_sorted = x[perm_idx]
        y_sorted = y[perm_idx]
        y_sorted = y_sorted.to('cuda:0')
        inputs = (x_sorted.to('cuda:0'), lengths)
        y_pred = model(inputs)
        loss_value = loss(y_pred, y_sorted)
        loss_value.backward()
        self.optimizer.step()
        return loss_value.item(), y_pred, y_sorted

class Evaluator:
    def __init__(self):
        pass

    def run(self, model, labels, dataset, batch_size=1):
        model.eval()
        valid_loader = DataLoader(dataset, batch_size=batch_size)
        cm = ConfusionMatrix(labels)
        for batch in valid_loader:
            y_pred, y_actual = self.inference(model, batch)
            _, best = y_pred.max(1)
            yt = y_actual.cpu().int().numpy()
            yp = best.cpu().int().numpy()
            cm.add_batch(yt, yp)
        return cm

    def inference(self, model, batch):
        with torch.no_grad():
            x, lengths, y = batch
            lengths, perm_idx = lengths.sort(0, descending=True)
            x_sorted = x[perm_idx]
            y_sorted = y[perm_idx]
            y_sorted = y_sorted.to('cuda:0')
            inputs = (x_sorted.to('cuda:0'), lengths)
            y_pred = model(inputs)
            return y_pred, y_sorted

def fit(model, labels, optimizer, loss, epochs, batch_size, train, valid, test):

    trainer = Trainer(optimizer)
    evaluator = Evaluator()
    best_acc = 0.0
    
    for epoch in range(epochs):
        print('EPOCH {}'.format(epoch + 1))
        print('=================================')
        print('Training Results')
        cm = trainer.run(model, labels, train, loss, batch_size)
        print('Validation Results')
        cm = evaluator.run(model, labels, valid)
        print(cm.get_all_metrics())
        if cm.get_acc() > best_acc:
            print('New best model {:.2f}'.format(cm.get_acc()))
            best_acc = cm.get_acc()
            torch.save(model.state_dict(), './checkpoint.pth')
    if test:
        model.load_state_dict(torch.load('./checkpoint.pth'))
        cm = evaluator.run(model, labels, test)
        print('Final result')
        print(cm.get_all_metrics())
    return cm.get_acc()

def whitespace_tokenizer(words: str) -> List[str]:
    return words.split() 

def sst2_tokenizer(words: str) -> List[str]:
    REPLACE = { "'s": " 's ",
                "'ve": " 've ",
                "n't": " n't ",
                "'re": " 're ",
                "'d": " 'd ",
                "'ll": " 'll ",
                ",": " , ",
                "!": " ! ",
                }
    words = words.lower()
    words = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", words)
    for k, v in REPLACE.items():
            words = words.replace(k, v)
    return [w.strip() for w in words.split()]


class Reader:

    def __init__(self, files, lowercase=True, min_freq=0,
                 tokenizer=sst2_tokenizer, vectorizer=None):
        self.lowercase = lowercase
        self.tokenizer = tokenizer
        build_vocab = vectorizer is None
        self.vectorizer = vectorizer if vectorizer else self._vectorizer
        x = Counter()
        y = Counter()
        for file_name in files:
            if file_name is None:
                continue
            with codecs.open(file_name, encoding='utf-8', mode='r') as f:
                for line in f:
                    words = line.split()
                    y.update(words[0])

                    if build_vocab:
                        words = self.tokenizer(' '.join(words[1:]))
                        words = words if not self.lowercase else [w.lower() for w in words]
                        x.update(words)
        self.labels = list(y.keys())

        if build_vocab:
            x = dict(filter(lambda cnt: cnt[1] >= min_freq, x.items()))
            alpha = list(x.keys())
            alpha.sort()
            self.vocab = {w: i+1 for i, w in enumerate(alpha)}
            self.vocab['[PAD]'] = 0

        self.labels.sort()

    def _vectorizer(self, words: List[str]) -> List[int]:
        return [self.vocab.get(w, 0) for w in words]

    def load(self, filename: str) -> TensorDataset:
        label2index = {l: i for i, l in enumerate(self.labels)}
        xs = []
        lengths = []
        ys = []
        with codecs.open(filename, encoding='utf-8', mode='r') as f:
            for line in f:
                words = line.split()
                ys.append(label2index[words[0]])
                words = self.tokenizer(' '.join(words[1:]))
                words = words if not self.lowercase else [w.lower() for w in words]
                vec = self.vectorizer(words)
                lengths.append(len(vec))
                xs.append(torch.tensor(vec, dtype=torch.long))
        x_tensor = torch.nn.utils.rnn.pad_sequence(xs, batch_first=True)
        lengths_tensor = torch.tensor(lengths, dtype=torch.long)
        y_tensor = torch.tensor(ys, dtype=torch.long)
        return TensorDataset(x_tensor, lengths_tensor, y_tensor)

Lets use the trec dataset again

In [0]:
!wget https://www.dropbox.com/s/08km2ean8bkt7p3/trec.tar.gz?dl=1
!tar -xzf 'trec.tar.gz?dl=1'

--2019-06-26 16:11:50--  https://www.dropbox.com/s/08km2ean8bkt7p3/trec.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/08km2ean8bkt7p3/trec.tar.gz [following]
--2019-06-26 16:11:50--  https://www.dropbox.com/s/dl/08km2ean8bkt7p3/trec.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc07ceac62805b51d07f217610fe.dl.dropboxusercontent.com/cd/0/get/AjmRZa8cUvcuVud0HfCm25tSr9rrBzwSvSRsT0YCnBmLX4ugEjzxDPrCXQdEZy6yHGcaBEzjLoQWBNMq9O8mFrLwnYbovya9ZcerCXxewB_lYg/file?dl=1# [following]
--2019-06-26 16:11:50--  https://uc07ceac62805b51d07f217610fe.dl.dropboxusercontent.com/cd/0/get/AjmRZa8cUvcuVud0HfCm25tSr9rrBzwSvSRsT0YCnBmLX4ugEjzxDPrCXQdEZy6yHGcaBEzjLoQWBNMq9O8mFrLwnYbovya9ZcerCXxewB_lYg/file?dl=1
Resolving uc07c

In [0]:
BASE = 'trec'
TRAIN = os.path.join(BASE, 'trec.nodev.utf8')
VALID = os.path.join(BASE, 'trec.dev.utf8')
TEST = os.path.join(BASE, 'trec.test.utf8')



r = Reader((TRAIN, VALID, TEST,), lowercase=False, vectorizer=bert_vectorizer, tokenizer=bert_tokenizer)
train = r.load(TRAIN)
valid = r.load(VALID)
test = r.load(TEST)

In [0]:

bert_small_dims = 768
batch_size = 50
epochs = 12
model = FineTuneClassifier(BERT_MODEL, len(r.labels), bert_small_dims)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters") 


model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(learnable_params, lr=1.0e-4)

fit(model, r.labels, optimizer, loss, epochs, batch_size, train, valid, test)

Model has 109486854 parameters
EPOCH 1
Training Results
{'acc': 0.8492, 'mean_precision': 0.8590416631560661, 'mean_recall': 0.784499104767959, 'macro_f1': 0.8093044230616595, 'weighted_precision': 0.8533096832063289, 'weighted_recall': 0.8492, 'weighted_f1': 0.848636551495934}
Validation Results
{'acc': 0.9292035398230089, 'mean_precision': 0.9095790095790096, 'mean_recall': 0.8812630270963604, 'macro_f1': 0.8914806712112505, 'weighted_precision': 0.9282319649576285, 'weighted_recall': 0.9292035398230089, 'weighted_f1': 0.9275440576086902}
New best model 0.93
EPOCH 2
Training Results
{'acc': 0.9628, 'mean_precision': 0.9524731788585094, 'mean_recall': 0.9448956967584476, 'macro_f1': 0.9484990826490849, 'weighted_precision': 0.9626990548505957, 'weighted_recall': 0.9628, 'weighted_f1': 0.9627032294356912}
Validation Results
{'acc': 0.9292035398230089, 'mean_precision': 0.8638110106824105, 'mean_recall': 0.8815288595309299, 'macro_f1': 0.871157144636212, 'weighted_precision': 0.93052016

0.966

We can see that this is a *massive* gain over our CNN baseline and also improves over our ELMo contextual embeddings for this dataset.  BERT has been shown high-performance results across many datasets, and integrating it into unstructured prediction problems is quite simple, as we saw in this section.

## Conclusion

In this section we investigated the Transformer model architecture, particularly in the context of pretraining LMs.  We discussed some of the model details and we looked at how BERT extends the GPT approach from OpenAI.  We then built our own fine-tuned classifier using the Hugging Face PyTorch library to create and re-load the BERT model and add our own layers on top.

